In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import time
import pyautogui
import math

# Load the YOLO11n-pose model
model = YOLO("yolo11n-pose.pt")

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Define keypoint indices for elbows (based on COCO 17-keypoint format)
LEFT_SHOULDER = 5
RIGHT_SHOULDER = 6
LEFT_ELBOW = 7
RIGHT_ELBOW = 8
LEFT_WRIST = 9
RIGHT_WRIST = 10

last_trigger_time = 0
cooldown = 1.5

# List to store elbow coordinates and timestamps
data = []

buffer = 50
cycle = ""

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform pose estimation
    results = model(frame, conf=0.5, classes=[0])  # Detect only person class

    # Get current timestamp
    timestamp = time.time()

    # Process results
    for result in results:
        if hasattr(result, 'keypoints') and result.keypoints is not None:
            keypoints = result.keypoints.xy.cpu().numpy()  # Shape: (num_persons, 17, 2)
            confidences = result.keypoints.conf.cpu().numpy()  # Shape: (num_persons, 17)

            for person_idx, (kpts, confs) in enumerate(zip(keypoints, confidences)):
                
                # Extract coordinates
                if confs[RIGHT_SHOULDER] > 0.5:
                    x, y = kpts[RIGHT_SHOULDER]
                    cv2.circle(frame, (int(x), int(y)), 8, (255, 0, 0), -1)  # Blue

                if confs[RIGHT_ELBOW] > 0.5:
                    x, y = kpts[RIGHT_ELBOW]
                    cv2.circle(frame, (int(x), int(y)), 8, (0, 255, 0), -1)  # Green

                if confs[RIGHT_WRIST] > 0.5:
                    x, y = kpts[RIGHT_WRIST]
                    cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), -1)  # Red

                if confs[LEFT_SHOULDER] > 0.5:
                    x, y = kpts[LEFT_SHOULDER]
                    cv2.circle(frame, (int(x), int(y)), 8, (255, 0, 0), -1)  # Blue

                if confs[LEFT_ELBOW] > 0.5:
                    x, y = kpts[LEFT_ELBOW]
                    cv2.circle(frame, (int(x), int(y)), 8, (0, 255, 0), -1)  # Green

                if confs[LEFT_WRIST] > 0.5:
                    x, y = kpts[LEFT_WRIST]
                    cv2.circle(frame, (int(x), int(y)), 8, (0, 0, 255), -1)  # Red

                # Draw Lines
                if confs[RIGHT_SHOULDER] > 0.5 and confs[RIGHT_ELBOW] > 0.5:
                    cv2.line(frame, tuple(kpts[RIGHT_SHOULDER].astype(int)), tuple(kpts[RIGHT_ELBOW].astype(int)), (100, 100, 255), 2)
                if confs[RIGHT_ELBOW] > 0.5 and confs[RIGHT_WRIST] > 0.5:
                    cv2.line(frame, tuple(kpts[RIGHT_ELBOW].astype(int)), tuple(kpts[RIGHT_WRIST].astype(int)), (100, 100, 255), 2)
                if confs[LEFT_SHOULDER] > 0.5 and confs[LEFT_ELBOW] > 0.5:
                    cv2.line(frame, tuple(kpts[LEFT_SHOULDER].astype(int)), tuple(kpts[LEFT_ELBOW].astype(int)), (100, 100, 255), 2)
                if confs[LEFT_ELBOW] > 0.5 and confs[LEFT_WRIST] > 0.5:
                    cv2.line(frame, tuple(kpts[LEFT_ELBOW].astype(int)), tuple(kpts[LEFT_WRIST].astype(int)), (100, 100, 255), 2)
                if confs[RIGHT_SHOULDER] > 0.5 and confs[LEFT_SHOULDER] > 0.5:
                    cv2.line(frame, tuple(kpts[RIGHT_SHOULDER].astype(int)), tuple(kpts[LEFT_SHOULDER].astype(int)), (100,100,255), 2)

                if confs[RIGHT_WRIST] > 0.5 and confs[RIGHT_SHOULDER] > 0.5 and confs[LEFT_WRIST] > 0.5 and confs[LEFT_SHOULDER] > 0.5:
                    if abs(int(kpts[RIGHT_SHOULDER][0]) - int(kpts[LEFT_SHOULDER][0])) > 100:
                        right_wrist_y = kpts[RIGHT_WRIST][1]
                        left_wrist_y = kpts[LEFT_WRIST][1]

                        shoulder_y = min(kpts[RIGHT_SHOULDER][1], kpts[LEFT_SHOULDER][1])

                        cv2.line(frame, (0, int(shoulder_y)-buffer), (frame.shape[1], int(shoulder_y)-buffer), (255,255,225), 4)

                        # Add pause between triggers
                        if time.time() - last_trigger_time > cooldown:
                            if left_wrist_y < shoulder_y - buffer:
                                pyautogui.press('right')
                                print("Next slide triggered")
                                last_trigger_time = time.time()
                                cycle = "Next"

                            elif right_wrist_y < shoulder_y - buffer:
                                pyautogui.press('left')
                                print("Previous slide triggered")
                                last_trigger_time = time.time()
                                cycle = "Previous"
                    else:
                        cv2.putText(frame, "Temp Pause", (30,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 0), 2, cv2.LINE_AA)

                if time.time() - last_trigger_time < 1:
                    cv2.putText(frame, cycle + " Slide Triggered", (30, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                1, (0, 255, 0), 2, cv2.LINE_AA)


    # Display the frame
    cv2.imshow("Elbow Tracking", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()